In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from clustering_utils import  build_cdlib_graph, build_ground_truth, get_ordered_communities
from utils import pre_processing_xes
import pm4py
import networkx as nx
from conversion import convert_to_dataframe
import pandas as pd

In [2]:
# # LOADING DATA
# # event log not partitioned (single trace with a list of events)
# event_log = xes_importer.apply('BP-Meets-IoT2022/sim_22d1p/EventLogXES_fixed_wc.xes') # ('sim_22d1p_22/EventLogXESNoSegment.xes')
# # event log partitioned (list of named traces)
# event_log_labelled = xes_importer.apply('BP-Meets-IoT2022/sim_22d1p/EventLogXES_fixed_wc.xes') # ('sim_22d1p_22/EventLogXES.xes')

In [ ]:
# LOADING DATA
# event log not partitioned (single trace with a list of events)
event_log = xes_importer.apply('BP-Meets-IoT2020/d21p1/EventLogXESNoSegment.xes') # ('sim_22d1p_22/EventLogXESNoSegment.xes')
# event log partitioned (list of named traces)
event_log_labelled = xes_importer.apply('BP-Meets-IoT2020/d21p1/EventLogXES.xes') # ('sim_22d1p_22/EventLogXES.xes')

In [4]:
event_log, event_log_labelled = pre_processing_xes(event_log, event_log_labelled)

In [5]:
dataframe = convert_to_dataframe(event_log_labelled)

In [6]:
dataframe= dataframe.sort_values(['time:timestamp', 'eventId'], ascending=[True, True])


In [7]:
dataframe

,concept:name,org:resource,time:timestamp,Activity,Resource,lifecycle:transition,eventId,case:concept:name,case:traceId
0,go_wardrobe,0,2020-03-16 00:00:00+00:00,go_wardrobe,0,complete,1484,prepared_sleep,117
1,get_clothes,0,2020-03-16 00:00:00+00:00,get_clothes,0,start,1485,prepared_sleep,117
2,get_clothes,0,2020-03-16 00:01:00+00:00,get_clothes,0,complete,1487,prepared_sleep,117
3,go_bathtub,0,2020-03-16 00:01:00+00:00,go_bathtub,0,complete,1488,prepared_sleep,117
4,have_bath,0,2020-03-16 00:01:00+00:00,have_bath,0,start,1489,prepared_sleep,117
...,...,...,...,...,...,...,...,...,...
260,brush_teeth,0,2020-04-05 23:31:00+00:00,brush_teeth,0,start,6171,prepared_sleep,117
261,brush_teeth,0,2020-04-05 23:37:00+00:00,brush_teeth,0,complete,6173,prepared_sleep,117
262,go_bed,0,2020-04-05 23:37:00+00:00,go_bed,0,complete,6174,prepared_sleep,117
263,sleep_in_bed,0,2020-04-05 23:37:00+00:00,sleep_in_bed,0,start,6175,prepared_sleep,117


In [8]:
dataframe.to_csv("abstraction/ground_truth/ground_truth_events-list.csv")

In [9]:
dataframe.dtypes

concept:name                         object
org:resource                         object
time:timestamp          datetime64[ns, UTC]
Activity                             object
Resource                             object
lifecycle:transition                 object
eventId                              object
case:concept:name                    object
case:traceId                         object
dtype: object

In [10]:
dataframe['time:timestamp'] = dataframe['time:timestamp'].dt.tz_localize(None)

In [11]:
dataframe.to_excel("abstraction/ground_truth/ground_truth_events-list.xlsx")

In [12]:
dataframe2 = dataframe.groupby(["case:concept:name", "time:timestamp"])


In [13]:
#dataframe.groupby(["case:concept:name"]).size()

In [14]:
dataframe2.first()


concept:name org:resource    Activity  \
case:concept:name time:timestamp                                              
Airing_home       2020-03-18 04:55:00   go_windows            0  go_windows   
                  2020-03-22 14:10:00   go_windows            0  go_windows   
                  2020-04-04 18:42:00   go_windows            0  go_windows   
Airing_home_end   2020-03-18 20:40:00   go_windows            0  go_windows   
                  2020-03-23 18:48:00   go_windows            0  go_windows   
...                                            ...          ...         ...   
wc                2020-04-05 19:38:00        go_wc            0       go_wc   
                  2020-04-05 19:39:00        wc_do            0       wc_do   
                  2020-04-05 19:40:00     wc_flush            0    wc_flush   
                  2020-04-05 21:05:00        go_wc            0       go_wc   
                  2020-04-05 21:07:00        wc_do            0       wc_do   

                                      Resource lifecycle:transition eventId  \
case:concept:name time:timestamp                                              
Airing_home       2020-03-18 04:55:00        0             complete    1902   
                  2020-03-22 14:10:00        0             complete    2931   
                  2020-04-04 18:42:00        0             complete    5424   
Airing_home_end   2020-03-18 20:40:00        0             complete    2083   
                  2020-03-23 18:48:00        0             complete    3107   
...                                        ...                  ...     ...   
wc                2020-04-05 19:38:00        0             complete    6044   
                  2020-04-05 19:39:00        0             complete    6047   
                  2020-04-05 19:40:00        0             complete    6050   
                  2020-04-05 21:05:00        0             complete    6109   
                  2020-04-05 21:07:00        0             complete    6112   

                                      case:traceId  
case:concept:name time:timestamp                    
Airing_home       2020-03-18 04:55:00          146  
                  2020-03-22 14:10:00          146  
                  2020-04-04 18:42:00          146  
Airing_home_end   2020-03-18 20:40:00          161  
                  2020-03-23 18:48:00          161  
...                                            ...  
wc                2020-04-05 19:38:00          127  
                  2020-04-05 19:39:00          127  
                  2020-04-05 19:40:00          127  
                  2020-04-05 21:05:00          127  
                  2020-04-05 21:07:00          127  

[1269 rows x 7 columns]

In [15]:
dfg_graph = dfg_discovery.apply(event_log)

In [16]:
G, nodes, nameIndex = build_cdlib_graph(dfg_graph)

In [17]:
ground_truth, ground_truth_dictionary =  build_ground_truth(event_log_labelled, nameIndex, G)

In [18]:
#dataframe = convert_to_dataframe(event_log_labelled)


In [19]:
#dataframe

In [20]:
comms_truth =[]
community_labels = []
idx = 0
for community_label in ground_truth_dictionary:
    #print(f"{idx} - {community_label}")
    comms_truth.append(list(ground_truth_dictionary[community_label]))
    community_labels.append(community_label)
    idx = idx + 1
      
struct_communities = get_ordered_communities(comms_truth, nodes, sort_events=False, community_labels=community_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 66]
[9, 10, 11, 12, 13]
[14, 15, 16, 17, 24]
[0, 8, 18, 19, 20, 21, 22, 23]
[25, 26]
[4, 27, 28, 29, 9, 30, 31, 32, 11, 33]
[34, 35, 36, 37, 41]
[38, 39, 40, 4, 27]
[42, 43, 44]
[45, 46]
[42, 47, 48]
[28, 49, 9, 50, 51, 52, 31, 53, 54, 55, 11, 33]
[0, 8, 18, 19, 56, 57, 22, 9, 58]
[11, 59]
[0, 8, 18, 19, 56, 60, 22, 23]
[61, 62]
[4, 27, 28, 63, 64, 65, 31, 53, 11, 33]


In [21]:
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.convert import convert_to_dataframe, convert_to_event_log
from pm4py.filtering import filter_event_attribute_values
from pm4py.objects.log.importer.xes import importer as xes_importer
from pandas.io.parsers.readers import read_csv
from pm4py.filtering import filter_event_attribute_values

In [26]:
df = convert_to_dataframe(event_log_labelled)

df= df.sort_values(['time:timestamp', 'eventId'], ascending=[True, True])
df.reset_index()

# add column 'case:concept:name' in log w\o segmentation
df['event:remove'] = 1
    
len_events = len(df.index)

In [27]:
ref_case_name = None
prev_row = None
event_day = None
traceId = -1

for index, row in df.iterrows():
    current_case_name = row['case:concept:name']
    print(f"{index} {current_case_name}")
    
    current_day = str(row['time:timestamp'])[0:10]
    if event_day != current_day:
        event_day = current_day
        traceId = traceId + 1
    
    if ref_case_name != current_case_name:
        if prev_row is not None:
            df.at[df.index[df['eventId'] == prev_row['eventId']][0], 'concept:name'] = ref_case_name
            df.at[df.index[df['eventId'] == prev_row['eventId']][0], 'lifecycle:transition'] = "complete"
            df.at[df.index[df['eventId'] == prev_row['eventId']][0], 'event:remove'] = 0
            
        ref_case_name = current_case_name
        df.at[df.index[df['eventId'] == row['eventId']][0], 'concept:name'] = current_case_name
        df.at[df.index[df['eventId'] == row['eventId']][0], 'lifecycle:transition'] = "start"
        df.at[df.index[df['eventId'] == row['eventId']][0], 'event:remove'] = 0
    
    df.at[df.index[df['eventId'] == row['eventId']][0], 'case:traceId'] = traceId   
    prev_row = row

0 prepared_sleep
1 prepared_sleep
2 prepared_sleep
3 prepared_sleep
4 prepared_sleep
5 prepared_sleep
6 prepared_sleep
7 prepared_sleep
8 prepared_sleep
9 prepared_sleep
10 prepared_sleep
11 prepared_sleep
12 prepared_sleep
13 prepared_sleep
14 prepared_sleep
15 prepared_sleep
265 drink
266 drink
267 drink
268 drink
269 drink
270 drink
271 drink
272 drink
1244 use_computer
1245 use_computer
1246 use_computer
1247 use_computer
1248 use_computer
1249 use_computer
1597 go_work
1598 go_work
1599 go_work
1600 go_work
1601 go_work
1602 go_work
1603 go_work
1604 go_work
1605 go_work
1606 go_work
1607 go_work
1608 go_work
1609 go_work
1610 go_work
1611 go_work
1612 go_work
1613 go_work
273 drink
274 drink
275 drink
276 drink
277 drink
278 drink
279 drink
280 drink
1250 use_computer
1251 use_computer
1252 use_computer
1253 use_computer
1254 use_computer
1255 use_computer
1872 someone_at_entrace
1873 someone_at_entrace
1874 someone_at_entrace
1934 eat_cold
1935 eat_cold
1936 eat_cold
1937 eat_co

In [28]:
df = df.drop(df[df['event:remove'] == 1].index)

In [29]:
df = df.drop(["event:remove"], axis=1)


In [30]:
df = df.drop(["case:traceId"], axis=1)

In [31]:
df = df.reset_index(drop=True)


In [32]:
idx = 0
for i in df.index:
     df.at[i, 'eventId'] = idx
     df.at[i, 'Activity'] = df.at[i, 'concept:name'] 
     df.at[i, 'case:concept:name'] = "case_id"
     idx = idx + 1

In [33]:
event_abstract = convert_to_event_log(df)

In [34]:
xes_exporter.apply(event_abstract, "abstraction/ground_truth/ground_truth_abstracted.xes")

exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
df.to_csv("abstraction/ground_truth/ground_truth_abstracted.csv")

In [36]:
df.dtypes


concept:name                         object
org:resource                         object
time:timestamp          datetime64[ns, UTC]
Activity                             object
Resource                             object
lifecycle:transition                 object
eventId                              object
case:concept:name                    object
dtype: object

In [37]:
df['time:timestamp'] = df['time:timestamp'].dt.tz_localize(None)

In [38]:
df.to_excel("abstraction/ground_truth/ground_truth_abstracted.xlsx")


In [43]:
for comm in struct_communities:
    current_community = comm["community_label"]
    evt_list = ",".join(comm["vals"])
    community_len = comm["len"]
    print(f"{current_community}:{community_len},{evt_list}")
   

cook_and_eat:12,go_fridge,get_ingredients_from_fridge,go_kitchen_shelf,get_ingredients_from_shelf,go_oven,use_oven,go_dining_table,eat_warm_meal,pack_food,put_meal_to_fridge,go_kitchen_sink,put_plate_to_sink
prepared_sleep:10,go_wardrobe,get_clothes,go_bathtub,have_bath,go_bathroom_sink,brush_teeth,go_bed,sleep_in_bed,change_clothes,go_to start
eat_cold:10,go_bathroom_sink,wash_hands,go_fridge,get_food_from_fridge,go_kitchen_shelf,get_bread,go_dining_table,eat_cold_meal,go_kitchen_sink,put_plate_to_sink
eat_warm:10,go_bathroom_sink,wash_hands,go_fridge,get_cold_warm_food,go_micro,use_micro,go_dining_table,eat_warm_meal,go_kitchen_sink,put_plate_to_sink
shop:9,go_wardrobe,change_clothes,go_shoe_shelf,dress_up_outdoor,go_outside,get_food,dress_down_outdoor,go_kitchen_shelf,pack_goods
go_work:8,go_wardrobe,change_clothes,go_shoe_shelf,dress_up_outdoor,go_workplace,work,dress_down_outdoor,finish_walk
do_walk:8,go_wardrobe,change_clothes,go_shoe_shelf,dress_up_outdoor,go_outside,walk_outsid